In [1]:
import torch  #类似于numpy

1、创建张量

In [5]:
x = torch.arange(12)  #x称为张量
x

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [14]:
print(x.shape)  #访问张量x的形状
x.numel()  #访问张量x中元素个数

torch.Size([12])


12

In [17]:
X = x.reshape(3,4)  #改变张量的形状
X

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [18]:
print(torch.zeros((1,2,3)))  #特定形状元素全为零的张量
torch.ones((1,2,3))  #特定形状元素全为一的张量

tensor([[[0., 0., 0.],
         [0., 0., 0.]]])


tensor([[[1., 1., 1.],
         [1., 1., 1.]]])

In [19]:
torch.randn(3,4)  #创建一个从标准正态分布（均值为零，标准差为1）中随机取样的张量

tensor([[-1.4411, -0.2720,  1.8260, -1.5867],
        [ 0.9430, -0.8197,  0.2119, -0.3287],
        [-1.5948,  1.1311,  0.5902,  1.5494]])

In [21]:
torch.tensor([[1,2,3],[4,5,6],[7,8,9]])  #创建给定值的张量

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

2、运算

In [31]:
#相同形状
x = torch.tensor([1.0,2,3,4])
y = torch.tensor([2,4,5,6])
x+y,x-y,x*y,x/y,x**y  #标准算数运算

(tensor([ 3.,  6.,  8., 10.]),
 tensor([-1., -2., -2., -2.]),
 tensor([ 2.,  8., 15., 24.]),
 tensor([0.5000, 0.5000, 0.6000, 0.6667]),
 tensor([1.0000e+00, 1.6000e+01, 2.4300e+02, 4.0960e+03]))

In [25]:
X = torch.arange(12,dtype=torch.float32).reshape((3,4))
Y = torch.tensor([[2.0,1,4,3],[3,4,5,6],[1,2,3,4]])
torch.cat((X,Y),dim=0)  #按行连结（将张量叠在一起）dim=0(按列)

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [ 2.,  1.,  4.,  3.],
        [ 3.,  4.,  5.,  6.],
        [ 1.,  2.,  3.,  4.]])

In [28]:
X == Y , X > Y , X < Y #逻辑运算符

(tensor([[False,  True, False,  True],
         [False, False, False, False],
         [False, False, False, False]]),
 tensor([[False, False, False, False],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True]]),
 tensor([[ True, False,  True, False],
         [False, False, False, False],
         [False, False, False, False]]))

In [30]:
X.sum()  #对张量中的所有元素进行求和，会产生一个单元素张量

tensor(66.)

In [32]:
#不同形状（广播机制：扩展数组，使张量具有相同形状
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a, b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

In [33]:
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

In [34]:
#节省内存
Z = torch.zeros_like(Y)  
print('id(Z):', id(Z))  
Z[:] = X + Y           #使用切片表示法将操作的结果分配给先前分配的数组
print('id(Z):', id(Z))

id(Z): 2759442549968
id(Z): 2759442549968


In [35]:
before = id(X)
X += Y          #在后续计算中没有重复使用X， 也可以使用X[:] = X + Y或X += Y来减少操作的内存开销。
id(X) == before

True

In [39]:
#将张量转换为numpy中的张量
A = X.numpy()
B = torch.tensor(A)
C = B.numpy()
type(A), type(B),type(C)

(numpy.ndarray, torch.Tensor, numpy.ndarray)

In [38]:
#张量转换为Python标量
a = torch.tensor([3.5])
a, a.item(), float(a), int(a)

(tensor([3.5000]), 3.5, 3.5, 3)

3、数据预处理

In [2]:
import os
                           #创建房价原数据
data_dir = 'D:/data'
os.makedirs(data_dir,exist_ok=True)
data_file = os.path.join(data_dir,'house_tiny.csv')
with open(data_file,'w') as f:
    f.write('NumRooms,Alley,Price\n') #列名
    f.write('NA,Pave,127500\n')
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [10]:
import pandas as pd

data = pd.read_csv(data_file)  #读入数据
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


In [6]:
inputs , outputs = data.iloc[:,0:2] , data.iloc[:,2]
inputs = inputs.fillna(inputs.mean(numeric_only=True))
inputs

,NumRooms,Alley
0,3.0,Pave
1,2.0,NaN
2,4.0,NaN
3,3.0,NaN


In [8]:
inputs = pd.get_dummies(inputs, dummy_na=True)   #将分类变量转换为虚拟变量
inputs = inputs.astype(int)       #转换数据类型
inputs

,NumRooms,Alley_Pave,Alley_nan
0,3,1,0
1,2,0,1
2,4,0,1
3,3,0,1


In [9]:
#将数值转换为张量（pytorch无法直接接受pandas的DataFrame,需先转换为numpy数组

X = torch.tensor(inputs.to_numpy(dtype=float))
Y = torch.tensor(outputs.to_numpy(dtype=float))
X , Y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))